Might need to install 

pip install -U gif

In [1]:
!pip install -U gif

  Using cached gif-23.3.0-py3-none-any.whl.metadata (2.9 kB)
Using cached gif-23.3.0-py3-none-any.whl (4.1 kB)


In [2]:
#!pip install -U gif
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os
import imageio
import datetime
from PIL import Image, ImageDraw
import gif

colors = [(1,1,1), (0, 1, 0), (1, 0, 0), (0, 0, 0)]  # White, green, red, black
n_bins = [4]  # Discretizes the interpolation into bins 
cmap_name = 'my_list' 
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=4)
threshold = 4  # max value


grid_height = 80
grid_width = 80
density_prob = 0.45
on_fire = 1
frames = []
percolated = 0


# set up

@gif.frame
def heat_map_plot(forest, threshold, cm):
    plt.imshow(forest, interpolation='nearest', vmax=threshold, cmap=cm)

def create_grid(grid_height, grid_width, density_prob):
    forest = np.zeros([grid_width, grid_height], dtype=int)
   
    for i in range(grid_width):
        for j in range(grid_height):
            tmp = random.random()
            if tmp < density_prob:
                forest[i,j]=1
    
    return forest

forest = create_grid(grid_width, grid_height, density_prob)

fig, ax = plt.subplots()

img = heat_map_plot(forest, threshold, cm)
plt.show()
frames.append(img)

#set fire to one side
for i in range(grid_width):
    if forest[0,i] ==1:
        forest[0,i]=2
img = heat_map_plot(forest, threshold, cm)
plt.show()
frames.append(img)


#propagate

def valid_cells(forest):
    '''
    Function which get all the coordinates of the grid
    :forest: the forest grid
    '''
    valid_cells = []
    for i in range(forest.shape[0]):
        for j in range(forest.shape[1]):
            valid_cells.append((i,j))
    return valid_cells


def propagate(forest, on_fire, percolated):
    new_forest = forest.copy()
    new_on_fire = 0
    perco = 0
    if percolated == 1:
        perco = 1
    valid = valid_cells(forest)
    
    if on_fire:
        for i in range(forest.shape[0]):
            for j in range(forest.shape[1]):
                if forest[i,j]==2:
                    new_forest[i,j]=3
                    #update neighbours
                    X = [-1, -1, 0, 1, 1, 1, 0, -1]
                    Y = [0, 1, 1, 1, 0, -1, -1, -1]
    
                    for k in range(len(X)):
                        temp_cell = (i + X[k], j + Y[k])
                        if temp_cell in valid and forest[i + X[k],j + Y[k]]==1:
                            new_forest[i + X[k],j + Y[k]] = 2
                            new_on_fire = 1
                            if i + X[k] == grid_height - 1:
                                perco = 1
               
    return new_forest, new_on_fire, perco
            
while on_fire == 1:
    
    forest, on_fire, percolated = propagate(forest, on_fire, percolated)

    img = heat_map_plot(forest, threshold, cm)
    plt.show()
    frames.append(img)
    
print(percolated)

#gif.save(frames, "My_forest_fire_1.gif", duration=1)

1


In [3]:
gif.save(frames, "My_forest_fire_1.gif", duration=1)